In [50]:
import pandas as pd
import altair as alt

#CO2 emission yearly change (metric tons)
climate_data = pd.read_csv('absolute-change-co2.csv')
climate_data = climate_data[climate_data['Year'] >= 1850]
countries = ['Asia', 'China', 'Africa', 'Sweden', 'Australia', 'Angola', 'Brazil', 'India', 'New Zealand', 'United States', 'Europe']
country_dfs = []
for i in range(len(countries)):
    country_i = climate_data.loc[climate_data['Entity'] == countries[i]]
    country_dfs.append(country_i)
climate_data = pd.concat(country_dfs)
climate_data.reset_index(inplace = True)
climate_data.drop(columns = ['index', 'Code'], inplace = True)
climate_data = climate_data.dropna()

In [51]:
#Yearly CO2 emissions per capita (metric tons)
climate_per_capita = pd.read_csv('co-emissions-per-capita.csv')
climate_per_capita = climate_per_capita[climate_per_capita['Year'] >= 1850]
countries = ['Asia', 'China', 'Africa', 'Sweden', 'Australia', 'Angola', 'Brazil', 'India', 'New Zealand', 'United States', 'Europe']
country_dfs = []
for i in range(len(countries)):
    country_i = climate_per_capita.loc[climate_per_capita['Entity'] == countries[i]]
    country_dfs.append(country_i)
climate_per_capita = pd.concat(country_dfs)
climate_per_capita.reset_index(inplace = True)
climate_per_capita.drop(columns = ['index', 'Code'], inplace = True)
total_data = pd.merge(climate_data, climate_per_capita, how ='left', on=['Entity', 'Year'])

In [52]:
#Adding in global life expectancy values
life_expect = pd.read_csv('life-expectancy.csv')
life_expect = life_expect[life_expect['Year'] >= 1850]
countries = ['Asia', 'China', 'Africa', 'Sweden', 'Australia', 'Angola', 'India', 'Brazil', 'New Zealand', 'United States', 'Europe']
country_dfs = []
for i in range(len(countries)):
    country_i = life_expect.loc[life_expect['Entity'] == countries[i]]
    country_dfs.append(country_i)
life_expect = pd.concat(country_dfs)
life_expect.reset_index(inplace = True)
life_expect.drop(columns = ['index', 'Code'], inplace = True)
life_expect = life_expect.dropna()
#Combining Life expect data into total_data with CO2 emissions
new_data = pd.merge(total_data, life_expect, how='left', on=['Entity', 'Year'])
# new_data['Life expectancy'] = new_data['Life expectancy'].fillna(0.0)
index_names = new_data[(new_data['Entity'] == 'Africa') & \
                                 (new_data['Per capita CO2 emissions'] == 0)].index
new_data.drop(index_names, inplace=True)


In [53]:
#Working on altair chart

dropdown = alt.binding_select(options=new_data["Entity"].unique(), name="Select a Country/Continent:")
selection = alt.selection(type="single", fields=["Entity"], bind=dropdown)

# Annual CO2 growth (abs)
# Per capita CO2 emissions
alt.Chart(new_data).mark_point(clip=True, filled=True).encode(
    alt.X('Year:O', axis = alt.Axis(title = 'Year'),
         scale=alt.Scale(zero=False)),
    alt.Y('Annual CO2 growth (abs)', axis = alt.Axis(title = 'Annual CO2 Emissions Change (Tonnes)')),
    alt.Color('Entity', scale=alt.Scale(scheme='category10')),
    alt.Size('Per capita CO2 emissions', title='Per Capita CO2 Emissions (t)'),
    opacity=alt.condition(selection, alt.value(1),alt.value(.2)),
    tooltip = ['Year', 'Entity', 'Annual CO2 growth (abs)', 'Per capita CO2 emissions']
).add_selection(
    selection
).properties(width=700, height=500, title='Annual CO2 Emissions Growth and Per Capita CO2 Emissions By Year'
).interactive()


alt.Chart(...)

In [54]:
#Life expectancy chart
selection = alt.selection(type='multi', fields=['Entity'], on='mouseover', nearest=True)

alt.Chart(new_data).mark_circle(filled=True).encode(
    x = 'Year:O',
    y = 'Life expectancy',
    color = alt.Color('Entity', scale = alt.Scale(scheme = 'spectral')),
    opacity=alt.condition(selection,alt.value(1),alt.value(.2)),
    tooltip = ['Entity', 'Life expectancy', 'Year', 'Per capita CO2 emissions'],
    size = alt.Size('Per capita CO2 emissions', title='Per Capita CO2 Emissions (t)')
).add_selection(
    selection
).properties(title='Life Expectancy Across Time and Per Capita CO2 Emissions', width=750, height=500)

alt.Chart(...)

In [55]:
#Global average temperature anomaly by year compared to 1961-1990 average global temp
global_temp = pd.read_csv('temperature-anomaly.csv')
# global_temp = global_temp[global_temp['Year'] >= 1900]
global_temp = global_temp[global_temp['Entity'] == 'Global']
global_temp.reset_index(inplace = True)
global_temp.drop(columns = ['index', 'Code'], inplace = True)

In [56]:
#Global temperature deviance chart
plot = alt.Chart(global_temp).mark_circle(filled=True).encode(
    alt.X('Year:O', scale=alt.Scale(zero = False)),
    alt.Y('Median temperature anomaly from 1961-1990 average', 
         axis=alt.Axis(title='Temperature Deviance °C')),
    tooltip=['Year', 'Median temperature anomaly from 1961-1990 average', 'Upper bound (95% CI)', 'Lower bound (95% CI)']
).properties(
    title={
        'text':['Median Global Temperature Deviance', 'Against the 1961-1990 Global Average Temperature with CI']
    },
    width=750,
    height=500).interactive()

bar = alt.Chart(global_temp).mark_errorbar(extent='ci', color='orange').encode(
    alt.X('Year:O'),
    alt.Y('Upper bound (95% CI)', 
          axis=alt.Axis(title='Temperature Deviance °C')),
    alt.Y2('Lower bound (95% CI)'),
).interactive()

plot + bar

alt.LayerChart(...)

In [57]:
co2_conc = pd.read_csv('co2-concentration-long-term.csv')
co2_conc_1850 = co2_conc[co2_conc['Year'] >= 1850]
total = alt.Chart(co2_conc).mark_circle(filled=True).encode(
    alt.X('Year:O',
         title='Year (Negative Values Represent Years BCE)'),
    alt.Y('CO2 concentrations (NOAA, 2018)',
        title='CO2 Concentrations (Parts Per Million)'),
    color = 'Entity',
    tooltip = ['Year', 'CO2 concentrations (NOAA, 2018)']).properties(
    width=800, 
    height=500,
    title='CO2 Concentrations in Parts Per Million Across Time: (803000 BCE - 2018 CE)').interactive()
co2_1850 = alt.Chart(co2_conc_1850).mark_circle(filled=True).encode(
    alt.X('Year:O'),
    alt.Y('CO2 concentrations (NOAA, 2018)',
         title='CO2 Concentrations (Parts Per Million)'),
    color = 'Entity',
    tooltip = ['Year', 'CO2 concentrations (NOAA, 2018)']).properties(
    width=800, 
    height=500,
    title='CO2 Concentrations in Parts Per Million Across Time: (1850 - 2018)').interactive()
alt.vconcat(total, co2_1850).resolve_scale(color='independent')

alt.VConcatChart(...)